# 說明
整理切好分批為 training / testing sets, 共切成 5 份

In [6]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import json

In [7]:
GTZAN_FOLDER = './gtzan_genre'
TRACKS_FOLDER = './track_by_genre'
SAVE_FOLDER = './genre_set'
SRC_FOLDER = '../fma_preprocessing'

In [8]:
with open('./category_label_mapping.json', 'r') as f:
    categoryMapping = json.load(f)
categoryMapping

{'0': 'blues',
 '1': 'classical',
 '2': 'country',
 '3': 'disco',
 '4': 'hiphop',
 '5': 'jazz',
 '6': 'metal',
 '7': 'pop',
 '8': 'reggae',
 '9': 'rock'}

In [9]:
iptPath = []
category = []
reverseCategoryMapping = {v: k for k, v in categoryMapping.items()}
for dirpath, _, filenames in os.walk(SRC_FOLDER):
    for f in filenames:
        cat = dirpath.split('/')[2]
        category.append(reverseCategoryMapping[cat])
        iptPath.append(f'../data{dirpath[2:]}/{f}')

In [10]:
dfNew = pd.concat([pd.Series(category), pd.Series(iptPath)], axis=1)
dfNew.columns = ['category', 'path']
dfNew.head()

,category,path
0,8,../data/fma_preprocessing/reggae/602-reggae___...
1,8,../data/fma_preprocessing/reggae/602-reggae___...
2,8,../data/fma_preprocessing/reggae/602-reggae___...
3,8,../data/fma_preprocessing/reggae/602-reggae___...
4,8,../data/fma_preprocessing/reggae/602-reggae___...


In [11]:
dfNew.to_csv('./all.csv', index=False)

## split into sets

In [12]:
BATCH_FOLDER = './input_set'
N_SPLIT = 5
BATCH_SIZE = round(dfNew.shape[0] / 5)
if not os.path.isdir(BATCH_FOLDER):
    os.makedirs(f'{BATCH_FOLDER}/raw', mode=0o777)
    os.makedirs(f'{BATCH_FOLDER}/training', mode=0o777)
    os.makedirs(f'{BATCH_FOLDER}/testing', mode=0o777)
for i in range(N_SPLIT):
    dfNew[i * BATCH_SIZE:(i + 1) * BATCH_SIZE].to_csv(f'{BATCH_FOLDER}/raw/raw-{i}.csv', index=False)
    print(dfNew[i * BATCH_SIZE:(i + 1) * BATCH_SIZE].head())
    print('batch ', i + 1, ' done')

  category                                               path
0        8  ../data/fma_preprocessing/reggae/602-reggae___...
1        8  ../data/fma_preprocessing/reggae/602-reggae___...
2        8  ../data/fma_preprocessing/reggae/602-reggae___...
3        8  ../data/fma_preprocessing/reggae/602-reggae___...
4        8  ../data/fma_preprocessing/reggae/602-reggae___...
batch  1  done
    category                                               path
638        4  ../data/fma_preprocessing/hiphop/21-hip_hop/00...
639        4  ../data/fma_preprocessing/hiphop/21-hip_hop/01...
640        4  ../data/fma_preprocessing/hiphop/21-hip_hop/00...
641        4  ../data/fma_preprocessing/hiphop/21-hip_hop/00...
642        4  ../data/fma_preprocessing/hiphop/21-hip_hop/01...
batch  2  done
     category                                               path
1276        0  ../data/fma_preprocessing/blues/3-blues/100487...
1277        0  ../data/fma_preprocessing/blues/3-blues/057799...
1278        0  ../d

## split training & testing sets

In [14]:
TESTING_RATIO = 0.1
for i, f in enumerate(os.listdir(f'{BATCH_FOLDER}/raw')):
    if '.ipynb_checkpoints' in f:
        continue
    print(i)
    df = pd.read_csv(f'{BATCH_FOLDER}/raw/{f}')
    trainingLen = round((1-TESTING_RATIO) * df.shape[0])
    testingLen = df.shape[0] - trainingLen
    df[:trainingLen].to_csv(f'{BATCH_FOLDER}/training/training-{i}.csv', index=False)
    df[trainingLen:].to_csv(f'{BATCH_FOLDER}/testing/testing-{i}.csv', index=False)

0
1
2
3
4
